In [77]:
import psycopg2
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Define connection parameters
db_params = {
    'dbname': 'pairs_trading',
    'user': 'postgres',
    'password': 'nishant',
    'host': 'localhost',
    'port': '5432'
}

print(db_params)


{'dbname': 'pairs_trading', 'user': 'postgres', 'password': 'nishant', 'host': 'localhost', 'port': '5432'}


In [78]:
# Establish connection
conn = None
try:
    conn = psycopg2.connect(**db_params)
    print("Connected to PostgreSQL database.")
    
    # Run the query and save the result in a DataFrame
    query = "SELECT symbol, timestamp, close FROM alpaca_data_hourly;"
    df = pd.read_sql_query(query, conn)
    # print(df)  # Display the DataFrame
except psycopg2.DatabaseError as e:
    print(f"Database connection failed: {e}")
finally:
    if conn:
        conn.close()
        print("Database connection closed.")

Connected to PostgreSQL database.


C:\Users\nisha\AppData\Local\Temp\ipykernel_7292\1808271386.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Database connection closed.


In [79]:
# for analysis lets take the following stocks
# ['AMD', 'META', 'NVDA', 'TSLA']

filtered_df_amd = df[df['symbol'].isin(['AMD'])]
filtered_df_meta = df[df['symbol'].isin(['META'])]
print (len(filtered_df_amd))
print (len(filtered_df_meta))

3469
3453


In [80]:
import pandas as pd

# Assuming 'common_column' is the name of the common column
common_column = 'timestamp'

# Identify rows in each DataFrame that are present in the other
rows_in_amd_also_in_meta = filtered_df_amd[common_column].isin(filtered_df_meta[common_column])
rows_in_meta_also_in_amd = filtered_df_meta[common_column].isin(filtered_df_amd[common_column])

# Filter the DataFrames to keep only the common rows
filtered_df_amd = filtered_df_amd[rows_in_amd_also_in_meta]
filtered_df_meta = filtered_df_meta[rows_in_meta_also_in_amd]

In [81]:
print (len(filtered_df_amd))
print (len(filtered_df_meta))

3453
3453


In [82]:
# Create a new DataFrame or use .loc to avoid "SettingWithCopyWarning"
filtered_df_amd=filtered_df_amd.copy()
filtered_df_meta=filtered_df_meta.copy()

# Apply log transformation using .loc
filtered_df_amd.loc[:, 'log_close'] = np.log(filtered_df_amd['close'])
filtered_df_meta.loc[:, 'log_close'] = np.log(filtered_df_meta['close'])

In [83]:
filtered_df_amd.head(5)

,symbol,timestamp,close,log_close
61582,AMD,2024-01-02 03:00:00-06:00,146.800,4.989071
61583,AMD,2024-01-02 04:00:00-06:00,146.640,4.987981
61584,AMD,2024-01-02 05:00:00-06:00,145.150,4.977768
61585,AMD,2024-01-02 06:00:00-06:00,144.100,4.970508
61586,AMD,2024-01-02 07:00:00-06:00,144.299,4.971888


In [84]:
X=filtered_df_meta['log_close'].reset_index(drop=True)
Y=filtered_df_amd['log_close'].reset_index(drop=True)

In [86]:
# Step 2: Run OLS regression
model = sm.OLS(Y, sm.add_constant(X)).fit()
residuals = model.resid

In [87]:
# Step 3: Test residuals for stationarity
adf_test = adfuller(residuals)

In [88]:
# Step 4: Interpret the result
print("ADF Test Statistic:", adf_test[0])
print("p-value:", adf_test[1])

# If p-value < 0.05, the residuals are stationary, and the series are cointegrated
if adf_test[1] < 0.05:
    print("The series are cointegrated.")
else:
    print("The series are not cointegrated.")

ADF Test Statistic: -2.412345041606113
p-value: 0.1382823843187277
The series are not cointegrated.
